In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages --------------------------------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ------------------------------------------------------------------------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")

In [3]:
dset_idx <- 1

In [4]:
p_thresh <- 0.05
q_thresh <- 0.1

In [5]:
univ_surv_df <- read_tsv(paste0(dirs$analysis_dir, "/survival/", unified_dsets[dset_idx], "_univ_survival_results.tsv"))
en_cph_df <- read_tsv(paste0(dirs$analysis_dir, "/survival/", unified_dsets[dset_idx], "_en_cph_results.tsv"))
network_mm_gs_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_gene_mm_gs.tsv"))
network_me_sig_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_eigengene_traits.tsv"))


-- Column specification --------------------------------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  km_pval = col_double(),
  km_qval = col_double(),
  cph_pval = col_double(),
  cph_qval = col_double(),
  coeff = col_double()
)



-- Column specification --------------------------------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  coeff = col_double()
)



-- Column specification --------------------------------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character(),
  module = col_character()
)
i Use `spec()` for the full column specifications.



-- Column specification --------------------------------------------------------------------------------------------------------------------------
cols(
  module = col_character(),
  f

# Univariate Survival

In [6]:
sig_univ_df <- univ_surv_df %>%
    dplyr::filter(km_pval < p_thresh | cph_pval < p_thresh) %>%
    dplyr::filter(km_qval < q_thresh | cph_qval < q_thresh)
# sig_univ_df

# Cox PH EN

In [7]:
# en_cph_df

# Joined data

In [8]:
univ_joined_df <- network_mm_gs_df %>%
    dplyr::filter(geneID %in% (sig_univ_df %>% pull(geneID))) %>%
    dplyr::select(geneID, module, mm_cor, mm_pval) %>%
    inner_join(network_me_sig_df, by = "module") %>%
    condense_figo(include_pvals = TRUE)

en_joined_df <- network_mm_gs_df %>%
    dplyr::filter(geneID %in% (en_cph_df %>% pull(geneID))) %>%
    dplyr::select(geneID, module, mm_cor, mm_pval) %>%
    inner_join(network_me_sig_df, by = "module") %>%
    condense_figo(include_pvals = TRUE)

In [9]:
length(union(univ_joined_df$geneID, en_joined_df$geneID))

[1] 94

In [10]:
# univ_joined_df
# en_joined_df

In [11]:
# Number of modules
nrow(network_me_sig_df)

# Unique modules in univ models
nrow(univ_joined_df)
n_univ_mod <- length(unique(univ_joined_df$module))
n_univ_mod
# prop module repeats
1 - n_univ_mod / nrow(univ_joined_df)

# Unique modules in EN model
nrow(en_joined_df)
n_en_mod <- length(unique(en_joined_df$module))
n_en_mod
# prop module repeats
1 - n_en_mod / nrow(en_joined_df)

[1] 11

[1] 73

[1] 9

[1] 0.8767123

[1] 43

[1] 11

[1] 0.744186